<a href="https://colab.research.google.com/github/smit-r/Shape-Generation-using-Spatially-Partitioned-Point-Clouds/blob/main/KDTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install open3d

In [2]:
import numpy as np
import scipy as scy
import os
import open3d
import numpy as np
import sklearn.decomposition

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/My Drive/Colab Notebooks/shapenet_chair_data/shapenet-chairs-pcd

/content/drive/My Drive/Colab Notebooks/shapenet_chair_data/shapenet-chairs-pcd


#### KDTree

In [4]:
## performs kdtree sorting on pointcloud
def kdtree(pointCloud, level):
  if (len(pointCloud)<=1):
    return pointCloud
  pointlist = np.array(pointCloud)
  plane = getPlane(level)
  median = getkthPosition(pointlist, plane, 0, len(pointlist)-1, int(len(pointlist)/2))
  leftArray = median[:int(len(median)/2)]
  rightArray = median[int((len(median)/2))+1:]
  leftSortedArray = kdtree(leftArray, level+1)
  rightSortedArray = kdtree(rightArray, level+1)
  return np.concatenate((leftSortedArray, [median[int(len(median)/2)]], rightSortedArray))

In [5]:
def getPlane(level):
  plane = level%3
  return plane

In [6]:
def getPartition(pointlist, axis, l, r):
  i = l
  x = pointlist[r, axis]
  for j in range(l, r):
    if pointlist[j,axis]<x:
      pointlist[[i,j]] = pointlist[[j,i]]
      i += 1
  pointlist[[i,r]] = pointlist[[r,i]]
  return i

In [7]:
def getkthPosition(pointlist, axis, l, r, k):
  i = getPartition(pointlist, axis, l, r)
  if (l>r):
    return np.array([])
  if (i == k):
    return pointlist
  elif (i == r):
    return getkthPosition(pointlist, axis, l, r-1, k)
  elif (i < k):
    return getkthPosition(pointlist, axis, i+1, r, k)
  else:
    return getkthPosition(pointlist, axis, l, i-1, k)

#### PCA Analysis

In [ ]:
## load the dataset
dataset = []
for f in os.listdir():
  if f[-4:]==".pcd":
    pcd = open3d.io.read_point_cloud(f)
    pointcloud = np.asarray(pcd.points)
    dataset.append(pointcloud)
  print(f)

In [ ]:
# sort using KDTree
data = np.array(dataset)
for i in range(len(dataset)):
  shape = data[i]
  shape = kdtree(shape, 0)
  data[i] = shape
  print(i)

In [ ]:
data = np.load('/content/drive/My Drive/shapenetdata.npy')

In [47]:
# construct matrix P
P = np.reshape(data, [5000,-1]) #(5000, 3000)
mean = np.mean(P, axis=0) #(5000, )
Pu = P - mean #P[i]-mean
Pu.shape

(5000, 3000)

In [4]:
# PCA
def pcaAnalysis(Pu):
  pca = sklearn.decomposition.PCA(n_components=100)
  pca.fit(Pu)
  comp = pca.components_
  return comp

In [48]:
comp = pcaAnalysis(Pu)

In [49]:
# transform and reconstruct
Pcomp = np.dot(Pu, comp.T)
print(Pcomp.shape)
Precon = np.dot(Pcomp, comp)
print(Precon.shape)
errors = Pu - Precon
errors = errors*errors #calculate errors for each shape
print(errors.shape)
errors = np.sum(errors, axis=1) #(5000, )
print(errors.shape)
errlist = [errors]

(5000, 100)
(5000, 3000)
(5000, 3000)
(5000,)


In [23]:
# sample two points randomly
def samplePoints():
  points = np.random.randint(0,1000, 2)
  return points

In [24]:
# swap points
def swap(shape, points):
  shape[3*points[0]], shape[3*points[1]] = shape[3*points[1]], shape[3*points[0]] # swap x
  shape[3*points[0]+1], shape[3*points[1]+1] = shape[3*points[1]+1], shape[3*points[0]+1] #swap y
  shape[3*points[0]+2], shape[3*points[1]+2] = shape[3*points[1]+2], shape[3*points[0]+2] #swap z
  return shape

In [25]:
# calculate error
def error(shape, shape_):
  err = shape - shape_
  err = np.dot(err.T, err)
  return err

In [ ]:
# ordering using PCA Reconstruction Error
# as given in paper
save_path = '/content/drive/My Drive/'
k = 10**4
count = 0
errs = errlist[0][0]
for s in range(len(Pu)):
  for i in range(k):
    shape = np.array(Pu[s])
    points = samplePoints() # sample two points
    shape = swap(shape, points)
    shapecomp = np.dot(shape.T, comp.T) # transform
    shaperecon = np.dot(shapecomp.T, comp) # inverse_transform
    err = error(shape, shaperecon)
    if err<errs:
      Pu[s] = shape
      count += 1
    else:
      continue
  print(count, s)
  comp = pcaAnalysis(Pu) # calculate new components
  #calculate error for next shape
  if s<len(Pu-1):
    shape = Pu[s+1]
    shapecomp = np.dot(shape.T, comp.T)
    shaperecon = np.dot(shapecomp.T, comp)
    errs = error(shape, shaperecon)
  if s%50==0 or s==len(Pu)-1: #calculate errors for each shape and store in errlist
    Pcomp = np.dot(Pu, comp.T)
    Precon = np.dot(Pcomp, comp)
    errors = Pu - Precon
    errors = errors*errors
    errors = np.sum(errors, axis=1)
    errlist.append(errors)
    np.save(save_path+'errlist', np.array(errlist))
    np.save(save_path+'Pu', Pu)

## results after performing swaping for 1000 shapes

[Reconstruction Error](https://drive.google.com/file/d/1MGAZNpFGKdh0b-mEuOIDiWvut3WGs_vA/view?usp=sharing)

In [26]:
def swapAll(Pu, Points):
  for i in range(len(points)):
    Pu[i] = swap(Pu[i], points[i])
  return Pu

In [27]:
def sampleAll():
  points = np.random.randint(0,1000,(5000,2))
  return points

In [28]:
def errAll(P_, comp):
  mean = np.mean(P_, axis=0)
  Pu_ = P_ - mean
  Pcomp = np.dot(Pu_, comp.T)
  Precon = np.dot(Pcomp, comp)
  errors = Pu_ - Precon
  errors = errors*errors
  errors = np.sum(errors, axis=1)
  return errors

In [ ]:
#### optimized trainig loop
save_path = '/content/drive/My Drive/'
for iter in range(1000):
  for k in range(10*4):
    P_ = np.array(P)
    points = sampleAll()
    P_ = swapAll(P_, points)
    # calculate error
    err = errAll(P_, comp)
    # compare
    bin = (err<errors).astype(int)
    binP_ = np.array([bin]*3000).T
    P = ((1-binP_)*P) + (binP_*P_) # select shapes to be swaped and swap them
  Pu = P - np.mean(P, axis=0)
  comp = pcaAnalysis(Pu)
  errors = errAll(P, comp)
  errlist.append(errors)
  np.save(save_path+'vec_errlist1', np.array(errlist))
  np.save(save_path+'vec_Pu1', P)
  print(iter, np.sum(errors))

## results after 400 iterations

[Reconstruction Error](https://drive.google.com/file/d/1-46E_roxPaFlTNoULcBjBYmD3uCfdoRH/view?usp=sharing)

#### GAN 

In [57]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
import numpy as np 
import matplotlib.pyplot as plt 
from tensorflow.keras.constraints import Constraint
from tensorflow.keras import backend

In [58]:
class ClipConstraint(Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value
 
	# clip model weights to hypercube
	def __call__(self, weights):
		return backend.clip(weights, -self.clip_value, self.clip_value)
 
	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}

In [59]:
const = ClipConstraint(0.01)

In [60]:
latent_dim = 100

Generator = keras.Sequential(
    [keras.Input(shape=(latent_dim,)),
     layers.Dense(100),
     layers.BatchNormalization(),
     layers.ReLU(),
     layers.Dense(100),
     layers.BatchNormalization(),
     layers.ReLU(),
     layers.Dense(100),
     layers.BatchNormalization(),
     layers.ReLU(),
     layers.Dense(100),
     layers.BatchNormalization(),
     layers.ReLU(),
     layers.Dense(100)
    ],
    name = 'Generator'
)

In [61]:
Discriminator = keras.Sequential(
    [keras.Input(shape=(latent_dim,)),
     layers.Dense(100, kernel_constraint=const),
     layers.BatchNormalization(),
     layers.LeakyReLU(alpha=0.2),
     layers.Dense(100, kernel_constraint=const),
     layers.BatchNormalization(),
     layers.LeakyReLU(alpha=0.2),
     layers.Dense(100, kernel_constraint=const),
     layers.BatchNormalization(),
     layers.LeakyReLU(alpha=0.2),
     layers.Dense(100, kernel_constraint=const),
     layers.BatchNormalization(),
     layers.LeakyReLU(alpha=0.2),
     layers.Dense(1)
    ],
    name = 'Discriminator'
)

In [54]:
def wass_loss_fn(label, coeff):
  wgloss = tf.reduce_mean(coeff*label)
  return wgloss

In [55]:
def loss_fn(real, fake):
  Eloss = tf.reduce_mean(real - fake, axis = 0)
  Sloss = tf.reduce_sum(tf.square(Eloss))
  return Sloss

In [63]:
## training loop

gen_opt = keras.optimizers.Adam()
dis_opt = keras.optimizers.Adam()

gloss = []
dlossr = []
dlossf = []

@tf.function
def train_step(shape_coeff, batch_size):
  ## sample noise vectors
  noise = tf.random.uniform(shape=[batch_size, 100], minval=-1, maxval=1) ## (batch_size, 100)
  ## get generator output
  gen_shape_coeff = Generator(noise) ## (batch_size, 100)
  label = -tf.ones([batch_size,1])
  #comb_shape_coeff = tf.concat([gen_shape_coeff, shape_coeff], axis=0) ## (2*batch_size, 100)
  ## generate labels
  #labels = tf.concat([tf.ones(gen_shape_coeff.shape[0],1), tf.zeros(shape_coeff.shape[0],1)], axis=0) ## (2*batch_size, 1)
  #labels += 0.05*tf.random.uniform(labels.shape) 

  ## train the discriminator
  with tf.GradientTape() as tape:
    predf = Discriminator(gen_shape_coeff) ## (2*batch_size, 1)
    d_loss_fake = wass_loss_fn(label, predf)
  grads = tape.gradient(d_loss_fake, Discriminator.trainable_weights)
  dis_opt.apply_gradients(zip(grads, Discriminator.trainable_weights))


  label = tf.ones([batch_size,1])
  shape_coeff_1 = shape_coeff[:batch_size]
  with tf.GradientTape() as tape:
    predr = Discriminator(shape_coeff_1)
    d_loss_real = wass_loss_fn(label, predr)
  grads = tape.gradient(d_loss_real, Discriminator.trainable_weights)
  dis_opt.apply_gradients(zip(grads, Discriminator.trainable_weights))

  #sample noise for gen training
  noise_g = tf.random.uniform(shape=[batch_size,100], minval=-1, maxval=1) ## (batch_size, 100)
  #get discriminator output for real data
  #dis_output = Discriminator(shape_coeff) ## (x1,x2,x3,x4,pred)
  labels = tf.ones([batch_size,1])
  shape_coeff_2 = shape_coeff[batch_size:]

  ## train the generator
  with tf.GradientTape() as tape:
    gen_coeff = Generator(noise_g)
    pred_g = Discriminator(gen_coeff) ## (x1,x2,x3,x4,pred)
    g_loss = loss_fn(shape_coeff_2, gen_coeff) # + wass_loss_fn(labels, pred_g)
  grads = tape.gradient(g_loss, Generator.trainable_weights)
  gen_opt.apply_gradients(zip(grads, Generator.trainable_weights))
  return d_loss_real, d_loss_fake, g_loss, gen_shape_coeff

In [64]:
Pcomp = Pcomp.astype("float32")

In [ ]:
## set betch_Size and epochs here and run this cell for training

batch_size = 2
dataset = tf.data.Dataset.from_tensor_slices(Pcomp)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)


epochs = 100
save_dir = ''


for epoch in range(epochs):
  print("\nstart epoch", epoch)
  for step, batch in enumerate(dataset):
    # train step
    d_loss_real, d_loss_fake, g_loss, gen_shape_coeff = train_step(batch, int(batch_size/2))
    gloss.append(g_loss)
    dlossr.append(d_loss_real)
    dlossf.append(d_loss_fake)

    #logging
    if step%50==0:
      print("discriminator loss at step %d:  d_loss_real : %.2f , d_loss_fake : %.2f" % (step, d_loss_real, d_loss_fake))
      print("generator loss at step %d: %.2f" % (step, g_loss))


      # save weights and gen_outputs

In [ ]:
plt.plot(np.arange(len(gloss)), gloss)

In [ ]:
plt.plot(np.arange(len(dlossf)), dlossf)

In [ ]:
plt.plot(np.arange(len(dlossr)), dlossr)

In [ ]:
gen_coeff = Generator(tf.random.uniform([1,100], -1, 1))
gen_coeff

In [70]:
gen_coeff = np.array(gen_coeff[0])

In [71]:
gen_shape = np.dot(gen_coeff, comp)
gen_shape = gen_shape + Pmean

In [93]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

In [95]:
pc0 = gen_shape
x = []
y = []
z = []
for i in range(1000):
  x.append(pc0[3*i])
  y.append(pc0[3*i + 1])
  z.append(pc0[3*i + 2])
x = np.asarray(x)
y = np.asarray(y)
z = np.asarray(z)

In [96]:
ax.scatter(x,z,y)

In [ ]:
fig

In [5]:
pc = np.load("/content/drive/My Drive/vec_Pu1.npy")

In [6]:
P = np.asarray(pc)
Pmean = np.mean(P, axis=0)
Pu = P - Pmean

In [7]:
comp = pcaAnalysis(Pu)
Pcomp = np.dot(Pu, comp.T)

In [8]:
Pcomp = Pcomp.astype('float32')